In [1]:
ls

biphenyl_mm_energy.ipynb
forcefield/
optimize.bak/
optimize.in
optimize.out
optimize.sav
optimize.tmp/
optimize4.bak/
optimize4.in
optimize4.out
optimize4.sav
optimize4.tmp/
optimize_1.3.0_starting.in
optimizehj.in
parametercheck.ipynb
plot_td_energies.py*
requirements.txt
result/
smirnoff_parameter_assignments.json
targets/
td_plot_data.pickle
td_targets_plots/
test_1.offxml
test_results.offxml
torsion_params_dataset_list_molecules.ipynb
z.py


In [2]:
import os
import json
import tempfile
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import (ForceField, 
UnassignedValenceParameterException, BondHandler, AngleHandler,
ProperTorsionHandler, ImproperTorsionHandler,
vdWHandler)


In [3]:
directory = '/Users/jessica/Downloads/release_1.2.0/fb-fit/targets/'
ff = '/Users/jessica/Documents/Grad_research/WBO_Torsions_ChayaPaper/release_1.3.0_2/fb-fit/fb-fit0/forcefield/test.offxml'
#sub_dir = [x[0] for x in os.walk(directory) if os.path.basename(x[0])[:2] == 'td']
sub_dir = [x[0] for x in os.walk(directory) if os.path.basename(x[0]).startswith('td')]

force_balance_file = 'optimize.in'

In [4]:
len(sub_dir)

710

In [5]:
# Find which line of optimize.in the targets specificaiton begins at
with open(force_balance_file, 'r') as f:
    replace_index = None
    force_balance_lines = f.readlines()
    for index,line in enumerate(force_balance_lines):
        try:
            #if 'name' in line[:4]:
            if line.startswith('name'):
                replace_index = index
                break
        except:
            pass
    #print(replace_index)
    

In [6]:
replace_index

69

In [ ]:
count = 0            
for asub in sub_dir:
    #do somehting with metadata
    metadata = os.path.join(asub,'metadata.json')
    #do something with mol2 file
    mol2 = os.path.join(asub, 'input.mol2')
    #get indices from json
    with open(metadata, 'r') as f:
        data = json.load(f)
        #print(data['attributes']['canonical_isomeric_explicit_hydrogen_mapped_smiles'])
        cmiles=data['attributes']['canonical_isomeric_explicit_hydrogen_mapped_smiles']
        dihedral=tuple(data['dihedrals'][0])
        #print(dihedral)
    
    #molecules =  Molecule.from_file(mol2, allow_undefined_stereo=True)
    molecules=Molecule.from_mapped_smiles(cmiles)
    topology = Topology.from_molecules([molecules])
    molecules.visualize()
    # Let's label using the Parsley force field
    forcefield = ForceField(ff, allow_cosmetic_attributes=True)

    # Run the molecule labeling
    molecule_force_list = forcefield.label_molecules(topology)
    #print(dict(molecule_force_list[0]['ProperTorsions']))
    # Print out a formatted description of the torsion parameters applied to this molecule
    for mol_idx, mol_forces in enumerate(molecule_force_list):
        for force_tag, force_dict in mol_forces.items():
            if force_tag == 'ProperTorsions':
                for (atom_indices, parameter) in force_dict.items():
                    #print('param id', parameter.id)
                    if (parameter.id == "TIG-fit0"):
                        #print('check1', parameter.id)
                        #check dihedral in forward or reverse order
                        if (atom_indices == dihedral) or (atom_indices == dihedral[::-1]):
                            print(atom_indices, dihedral)
                            print(asub.split('/')[-1])

                            print('check2')
                            #run forcebalance
                            tmp_file = tempfile.NamedTemporaryFile(suffix='.in')
                            if 0:
                                with open(tmp_file.name,'w+t') as f1:
                                    for index,line in enumerate(force_balance_lines):
                                        if index != replace_index:
                                            print(index, line)
                                            f1.write(line)
                                        else:
                                            print('original', line)
                                            print('replace ', line.split()[0] + ' ' + os.path.basename(asub))
                                            f1.write(line.split()[0] + ' ' + os.path.basename(asub)+"\n")
                                    f1.flush()
                                    f1.seek(0)
                                    print('ForceBalance ' + f1.name)
                                    print(os.path.isfile(f1.name))
                                    print(os.system('ForceBalance ' +f1.name))
                                    print('done')

                                    import time
                                    time.sleep(1000)
                            if 1:
                                with open('temp.in','w') as f1:
                                    for index,line in enumerate(force_balance_lines):
                                        if index != replace_index:
                                            print(index, line)
                                            f1.write(line)
                                        else:
                                            print('original', line)
                                            print('replace ', line.split()[0] + ' ' + os.path.basename(asub))
                                            f1.write(line.split()[0] + ' ' + os.path.basename(asub)+"\n")
                                            
                                print('ForceBalance ' + 'debug.in')
                                print(os.system('ForceBalance ' + 'debug.in'))
                                print('done')
                                time.sleep(1000)

                            #make plot
                            
                            #count += 1
                            #params.append(parameter.id)
                            #print(index)

    
        

(1, 3, 5, 16) (1, 3, 5, 16)
td_OpenFF_Gen_2_Torsion_Set_4_eMolecules_Discrepancy_2_115_C15H17NO3
check2
0 # ForceBalance input file generated by MakeInputFile.py

1 # The octothorpe '#' is a comment symbol

2 # Note: If the specified value is 'None' then the option will truly be set to None - not the string 'None'

3 # Note: 'Section' option types are more complicated and may require you to read the documentation

4 # Note: Boolean option types require no value, the key being present implies 'True'

5 

6 $options

7 # (string) Directory containing force fields, relative to project directory

8 ffdir forcefield

9 

10 # (string) Type of the penalty, L2 or L1 in the optimizer

11 penalty_type L2

12 

13 #rmonicBondForce/Bond/k:: (allcap) The job type, defaults to a single-point evaluation of objective function

14 jobtype optimize

15 

16 # (list) The names of force fields, corresponding to directory forcefields/file_name.(itp|gen)

17 forcefield test.offxml # openff_unconstrained-1.

0
done


In [ ]:
molecules.visualize(backend='openeye')


In [15]:
ls targets/

err_bonds_changed/
error_mol2s/
optgeo_OpenFF_Gen_2_Opt_Set_1_Roche-0/
optgeo_OpenFF_Gen_2_Opt_Set_1_Roche-1/
optgeo_OpenFF_Gen_2_Opt_Set_1_Roche-2/
optgeo_OpenFF_Gen_2_Opt_Set_1_Roche-3/
optgeo_OpenFF_Gen_2_Opt_Set_1_Roche-4/
optgeo_OpenFF_Gen_2_Opt_Set_1_Roche-5/
optgeo_OpenFF_Gen_2_Opt_Set_2_Coverage-0/
optgeo_OpenFF_Gen_2_Opt_Set_2_Coverage-1/
optgeo_OpenFF_Gen_2_Opt_Set_2_Coverage-2/
optgeo_OpenFF_Gen_2_Opt_Set_2_Coverage-3/
optgeo_OpenFF_Gen_2_Opt_Set_2_Coverage-4/
optgeo_OpenFF_Gen_2_Opt_Set_2_Coverage-5/
optgeo_OpenFF_Gen_2_Opt_Set_2_Coverage-6/
optgeo_OpenFF_Gen_2_Opt_Set_2_Coverage-7/
optgeo_OpenFF_Gen_2_Opt_Set_3_Pfizer-0/
optgeo_OpenFF_Gen_2_Opt_Set_3_Pfizer-1/
optgeo_OpenFF_Gen_2_Opt_Set_3_Pfizer-2/
optgeo_OpenFF_Gen_2_Opt_Set_3_Pfizer-3/
optgeo_OpenFF_Gen_2_Opt_Set_4_eMolecules_Discrepancy-00/
optgeo_OpenFF_Gen_2_Opt_Set_4_eMolecules_Discrepancy-01/
optgeo_OpenFF_Gen_2_Opt_Set_4_eMolecules_Discrepancy-02/
optgeo_OpenFF_Gen_2_Opt_Set_4_eMolecules_Discrepancy-03/
optgeo_Ope